In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.python.keras.datasets import imdb
from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.5.0


In [2]:
wiki_read = pd.read_csv('simple-wikipedia-sentences-10-article.csv')
print(wiki_read.head(10))

vocabuloryFile = open('simple-engish-wiki-10-article-vocabulory.txt','r')
fileContent = vocabuloryFile.read()
vocabulory_read = fileContent.split('\n')
#print(vocabulory_read[0])  
vocabuloryFile.close()

vocab_to_id_dict = dict(zip(vocabulory_read, range(len(vocabulory_read))))
#print(vocab_to_id_dict)
id_to_vocab_dict = dict(zip(range(len(vocabulory_read)), vocabulory_read))
#print(id_to_vocab_dict)

   class                                           sentence
0      1  Roanoke Island is an island that is part of Da...
1      1  Roanoke was first settled around 8,000 B.C. Vi...
2      1  At time of European Contact the island was hom...
3      1  It was located on what is now the coast of Nor...
4      1  Walter Raleigh and others lived there from 158...
5      1  It was eventually given up, and nobody knows why.
6      1  Virginia Dare, the first English child born in...
7      1  There was a Civil War battle fought on this is...
8      1  Union forces had the advantage because they ou...
9      1  The Confederate Army beveling that the swamps ...


In [4]:
train_features = []
train_labels = []
for index, row in wiki_read.iterrows():
    train_features.append([vocab_to_id_dict[word] for word in row['sentence'].split(' ')])
    train_labels.append(row['class'])

# test_features = train_features[10:200]
# test_labels = train_labels[10:200]

test_features = train_features
test_labels = train_labels

train_features = np.array(train_features)
train_labels = np.array(train_labels)

test_features = np.array(test_features)
test_labels = np.array(test_labels)

print(test_features)
print(test_labels)

vocab_size = len(vocabulory_read)
sentence_size = 25
embedding_size = 50
model_dir = tempfile.mkdtemp()

# we assign the first indices in the vocabulary to special tokens that we use
# for padding, as start token, and for indicating unknown words
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2

print("Pad sequences (samples x time)")
x_train = sequence.pad_sequences(train_features, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
y_train = train_labels
x_test = sequence.pad_sequences(test_features, 
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post', 
                                value=pad_id)

y_test = test_labels
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

[list([246, 826, 3163, 4423, 4347, 12, 3163, 2709, 2765, 44, 2540])
 list([246, 1373, 2412, 863, 3400, 1601, 4260, 328, 113, 3982, 2443, 2784, 4332, 3400, 3274, 4347, 348, 3274, 2411, 377])
 list([4148, 1516, 2765, 432, 1257, 3274, 4347, 1373, 3694, 3065, 629, 2636, 2951, 1373, 4423, 3785, 950, 721, 3979, 558, 4474, 3274, 1641, 4142])
 list([2402, 1373, 1077, 4020, 3979, 3163, 3882, 3274, 2737, 2765, 1028, 4116])
 list([3259, 2718, 3956, 2508, 2527, 3194, 2701, 2155, 3065, 310])
 list([2402, 1373, 2142, 933, 240, 3956, 571, 1910, 450])
 list([4249, 1228, 3274, 2412, 1030, 1711, 3285, 721, 3274, 3458, 4176, 1373, 3285, 4020, 561])
 list([2951, 1373, 4029, 2251, 1222, 734, 4000, 4020, 869, 4347, 721, 2662, 2765, 996])
 list([1362, 4509, 2406, 3274, 1143, 465, 4058, 3317, 3274, 4409, 3240])
 list([458, 4409, 1790, 2599, 12, 3274, 2655, 4020, 3274, 4347, 4332, 2250, 3336, 2976, 576, 1873, 1941])
 list([1362, 2423, 3422, 1646, 3274, 4409, 1296, 4020, 3192, 423, 371, 3274, 4409, 4379, 759, 4

In [5]:
x_len_train = np.array([min(len(x), sentence_size) for x in train_features])
x_len_test = np.array([min(len(x), sentence_size) for x in test_features])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(train_features))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [6]:
all_classifiers = {}
def train_and_evaluate(classifier):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=2500)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()
#     # Un-comment code to download experiment data from Colaboratory
#     from google.colab import files
#     model_name = os.path.basename(os.path.normpath(classifier.model_dir))
#     ! zip -r {model_name + '.zip'} {classifier.model_dir}
#     files.download(model_name + '.zip')

In [7]:
head = tf.contrib.estimator.binary_classification_head()

def lstm_model_fn(features, labels, mode):    
    # [batch_size x sentence_size x embedding_size]
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=tf.random_uniform_initializer(-1.0, 1.0))

    # create an LSTM cell of size 100
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)
    
    # create the complete LSTM
    _, final_states = tf.nn.dynamic_rnn(
        lstm_cell, inputs, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states.h

    logits = tf.layers.dense(inputs=outputs, units=1)

    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    optimizer = tf.train.AdamOptimizer()

    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)


lstm_classifier = tf.estimator.Estimator(model_fn=lstm_model_fn,
                                         model_dir=os.path.join(model_dir, 'lstm'))
train_and_evaluate(lstm_classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplkl8o7p7/lstm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f966ca84b00>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplkl8o7p7/lstm/model.ckpt.
INFO:tensorflow:loss = 69.73157, step = 1
INFO:tensorflow:global_step/sec: 8.74257
INFO:tensorflow:loss = 7.4740233, step = 101 (11.439 sec)
INFO:tensorflow:global_step/sec: 10.6026
INFO:tensorflow:loss = 2.9051247, step = 201 (9.431 sec)
INFO:tensorflow:global_step/sec: 11.1411
INFO:tensorf